In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

import re
from sklearn import preprocessing

Prostranstvo = pd.read_excel("Data.xlsx", sheet_name="Prostranstvo")
Dosug = pd.read_excel("Data.xlsx", sheet_name="Dosug")
Themes = pd.read_excel("Data.xlsx", sheet_name="Themes")
Format = pd.read_excel("Data.xlsx", sheet_name="Format")
Motivi = pd.read_excel("Data.xlsx", sheet_name="Motivi")

S_Prostranstvo = pd.read_excel("S_Data.xlsx", sheet_name="Prostranstvo")
S_Dosug = pd.read_excel("S_Data.xlsx", sheet_name="Dosug")
S_Themes = pd.read_excel("S_Data.xlsx", sheet_name="Themes")
S_Format = pd.read_excel("S_Data.xlsx", sheet_name="Format")
S_Motivi = pd.read_excel("S_Data.xlsx", sheet_name="Motivi")

Prostranstvo = pd.concat([Prostranstvo, S_Prostranstvo], axis=0)
Dosug = pd.concat([Dosug, S_Dosug], axis=0)
Themes = pd.concat([Themes, S_Themes], axis=0)
Format = pd.concat([Format, S_Format], axis=0)
Motivi = pd.concat([Motivi, S_Motivi], axis=0)

Prostranstvo.reset_index(drop=True, inplace=True)
Dosug.reset_index(drop=True, inplace=True)
Themes.reset_index(drop=True, inplace=True)
Format.reset_index(drop=True, inplace=True)
Motivi.reset_index(drop=True, inplace=True)

In [70]:
def recode_empty_cells(dataframe, list_of_columns):

    for column in list_of_columns:
      dataframe[column] = dataframe[column].replace(r'\s+', np.nan, regex=True)
      dataframe[column] = dataframe[column].fillna(0)

    return dataframe

Dosug = Dosug.drop(['Unnamed: 0', 'Stand Up', 'Муз. группа', 'Фотозона', 'Инф.стойки', 'Перекус', 'Тишина', 'Знакомства', 'СКО', 'среднее'], axis=1)
Dosug.rename(columns = {'Stand Up норм.': 'Stand Up', 'Муз. группа норм.': 'Муз. группа', 'Фотозона норм.': 'Фотозона', 'Инф.стойки норм.': 'Инф.стойки', 'Перекус норм.': 'Перекус', 'Тишина норм.': 'Тишина', 'Знакомства норм.': 'Знакомства'}, inplace = True)

Format = Format.drop(['Unnamed: 0', 'Диалог на равных', 'Мастер-класс', 'Дискуссия', 'Форсайт', 'Выступление', 'Кейс', 'СКО', 'среднее'], axis=1)
Format.rename(columns = {'Диалог на равных норм.': 'Диалог на равных', 'Мастер-класс группа норм.': 'Мастер-класс', 'Дискуссия норм.': 'Дискуссия', 'Форсайт норм.': 'Форсайт', 'Выступление норм.': 'Выступление', 'Кейс норм.': 'Кейс'}, inplace = True)

Themes = Themes.drop(['Unnamed: 0', 'Старт карьеры', 'Инвестиции beginner', 'Инвестиции intermediate', 'Стартапы', 'Креативные индустрии', 'Импортозамещение в технологиях', 'Санкции и возможности', 'IT', 'Soft skills', 'Профессии будущего', 'Психология и саморазвитие', 'Фин.грамотность', 'Социальное предпринимательство', 'Мировая экономика', 'География карьеры', 'СКО', 'среднее'], axis=1)
Themes.rename(columns = {'Старт карьеры норм.': 'Старт карьеры', 'Инвестиции beginner норм.': 'Инвестиции beginner', 'Инвестиции intermediate норм.': 'Инвестиции intermediate', 'Стартапы норм.': 'Стартапы', 'Креативные индустрии норм.': 'Креативные индустрии', 'Импортозамещение в технологиях норм.': 'Импортозамещение в технологиях', 'Санкции и возможности норм.': 'Санкции и возможности', 'IT норм.': 'IT', 'Soft skills норм.': 'Soft skills', 'Профессии будущего норм.': 'Профессии будущего', 'Психология и саморазвитие норм.': 'Психология и саморазвитие', 'Фин.грамотность норм.': 'Фин.грамотность', 'Социальное предпринимательство норм.': 'Социальное предпринимательство', 'Мировая экономика': 'Мировая экономика', 'География карьеры': 'География карьеры'}, inplace = True)

Motivi = Motivi.drop(['Unnamed: 0', 'Знакомства', 'Поиск работы', 'Карьерная траектория', 'Новые ЗУНы', 'Участие в масштабном мероприятии', 'Спикер', 'Понимание экономической ситуации', 'Отдых с друзьями', 'Альтернатива занятиям', 'СКО', 'среднее'], axis=1)
Motivi.rename(columns = {'Знакомства норм.': 'Знакомства', 'Поиск работы норм.': 'Поиск работы', 'Карьерная траектория норм.': 'Карьерная траектория', 'Новые ЗУНы норм.': 'Новые ЗУНы', 'Участие в масштабном мероприятии норм.': 'Участие в масштабном мероприятии', 'Спикер норм.': 'Спикер', 'Понимание экономической ситуации норм.': 'Понимание экономической ситуации', 'Отдых с друзьями норм.': 'Отдых с друзьями', 'Альтернатива занятиям норм.': 'Альтернатива занятиям'}, inplace = True)


def rule(x, y):
    if x == y:
         return x
    else:
         return y

Dosug['выбор_1'] = Dosug.iloc[:, 9:16].T.apply(lambda x: x.nlargest(1).idxmin())
Dosug['выбор_2'] = Dosug.iloc[:, 9:16].T.apply(lambda x: x.nlargest(3).idxmin())

Format['выбор_1'] = Format.iloc[:, 9:15].T.apply(lambda x: x.nlargest(1).idxmin())
Format['выбор_2'] = Format.iloc[:, 9:15].T.apply(lambda x: x.nlargest(3).idxmin())

Themes['выбор'] = Themes.iloc[:, 9:25].T.apply(lambda x: x.nlargest(1).idxmin())

Motivi['выбор'] = Motivi.iloc[:, 9:19].T.apply(lambda x: x.nlargest(1).idxmin())


Dosug['выбор'] = Dosug.apply(lambda x: rule(x['выбор_1'], x['выбор_2']), axis =  1)
Dosug = Dosug.drop(['выбор_1', 'выбор_2'], axis=1)

Format['выбор'] = Format.apply(lambda x: rule(x['выбор_1'], x['выбор_2']), axis =  1)
Format = Format.drop(['выбор_1', 'выбор_2'], axis=1)


In [71]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('All_Data.xlsx')

# Write each dataframe to a different worksheet.
Prostranstvo.to_excel(writer, sheet_name='Prostranstvo')
Dosug.to_excel(writer, sheet_name='Dosug')
Themes.to_excel(writer, sheet_name='Themes')
Format.to_excel(writer, sheet_name='Format')
Motivi.to_excel(writer, sheet_name='Motivi')

# Close the Pandas Excel writer and output the Excel file.
writer.save()